In [114]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



In [115]:
data = pd.read_csv('news.csv')

In [116]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
#split the training sets and testing sets

train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [5]:
# process the headline into string, for later convinience

trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [6]:
#convert the training set into matrix form

basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31675)


In [7]:
#first, we will try to use logistic regreesion since this is a classification problem

LogicModel = LogisticRegression()
LogicModel = LogicModel.fit(basictrain, train["Label"])


In [12]:
#converting the est lines into string form

testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)


In [13]:
#using sklearn library to check the accuracy

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

predictions1 = LogicModel.predict(basictest)
accuracy1=accuracy_score(test['Label'], predictions1)


In [14]:
print('Logic Regression 1 accuracy: ', accuracy1)

Logic Regression 1 accuracy:  0.425925925926


In [15]:
#not so good

In [17]:
#check the fitted parameters

basicwords = basicvectorizer.get_feature_names()
basiccoeffs = LogicModel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])


,Coefficient,Word
19419,0.497924,nigeria
25261,0.452526,self
29286,0.428011,tv
15998,0.425863,korea
20135,0.425716,olympics


In [18]:
coeffdf.head(5)

,Coefficient,Word
19419,0.497924,nigeria
25261,0.452526,self
29286,0.428011,tv
15998,0.425863,korea
20135,0.425716,olympics


In [19]:
coeffdf.tail(5)

,Coefficient,Word
16949,-0.463116,low
3651,-0.470454,begin
25433,-0.494555,sex
24754,-0.549725,sanctions
24542,-0.587794,run


In [20]:
#we will try logistic model again, but this time
#we will set trstriction on training set

advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [21]:
print(advancedtrain.shape)

(1611, 657)


In [22]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [24]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

advancedtest = advancedvectorizer.transform(testheadlines)


In [25]:
#making predictions

preds2 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds2)

In [26]:
print('Logic Regression 2 accuracy: ', acc2)

Logic Regression 2 accuracy:  0.571428571429


In [27]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])


,Coefficient,Words
411,1.264377,set to
31,1.259669,and other
391,1.216202,right to
276,1.166895,likely to
14,1.128663,after the


In [28]:
advcoeffdf.head(5)

,Coefficient,Words
411,1.264377,set to
31,1.259669,and other
391,1.216202,right to
276,1.166895,likely to
14,1.128663,after the


In [29]:
advcoeffdf.tail(5)

,Coefficient,Words
131,-1.098193,fire on
40,-1.128193,around the
366,-1.144952,phone hacking
597,-1.146688,up in
452,-1.393351,the country


In [30]:
#Naive Bayes is usually working well on nltk issue, give it a shot

advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)


In [31]:
print(advancedtrain.shape)

(1611, 529)


In [33]:
#import naive bayes library - we have multiple classes
from sklearn.naive_bayes import MultinomialNB

advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])


In [34]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))


In [35]:
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)


In [36]:
acc4=accuracy_score(test['Label'], preds4)

In [38]:
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.513227513228


In [39]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Coefficient,Words
234,-5.180422,israel
497,-5.252936,was
94,-5.312797,china
318,-5.317049,not
394,-5.329156,says


In [40]:
advcoeffdf.tail(5)

,Coefficient,Words
204,-6.948273,held
446,-6.951188,tells
426,-6.952040,story
290,-6.970522,monday
268,-6.970642,lost


In [41]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.5, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [42]:
print(advancedtrain.shape)

(1611, 654)


In [43]:
advancedmodel = MultinomialNB(alpha=0.0001)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])


In [44]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))


In [45]:
advancedtest = advancedvectorizer.transform(testheadlines)
preds5 = advancedmodel.predict(advancedtest)


In [46]:
acc5 = accuracy_score(test['Label'], preds5)

In [47]:
print('NBayes 2 accuracy: ', acc5)

NBayes 2 accuracy:  0.537037037037


In [48]:
advcoeffdf.head(5)

,Coefficient,Words
234,-5.180422,israel
497,-5.252936,was
94,-5.312797,china
318,-5.317049,not
394,-5.329156,says


In [49]:
advcoeffdf.tail(5)

,Coefficient,Words
204,-6.948273,held
446,-6.951188,tells
426,-6.952040,story
290,-6.970522,monday
268,-6.970642,lost


In [74]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


X_train shape: (1611, 613)
X_test shape: (1134, 4565)


In [75]:
#Finally, we can try deep learning model as well,
#Since this is string classification, LSTM is best option
#set up the parameters


In [77]:
from keras.preprocessing import sequence

print('Pad sequences (samples x time)')

X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Pad sequences (samples x time)


In [78]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [79]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (1611, 200)
X_test shape: (1134, 200)


In [85]:
#now we can finally add layers to the model

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.core import Dense, Dropout, Activation, Lambda




/Users/taocheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  if __name__ == '__main__':
/Users/taocheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  # Remove the CWD from sys.path while we load stuff.


In [88]:
batch_size = 32
nb_classes = 2
advancedvectorizer = TfidfVectorizer( min_df=0.04, max_df=0.3, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
print(advancedtrain.shape)

(1611, 401)


In [89]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

X_train shape: (1611, 401)
X_test shape: (378, 401)


In [90]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


In [91]:
scale = np.max(X_train)
X_train /= scale
X_test /= scale

In [92]:
mean = np.mean(X_train)
X_train -= mean
X_test -= mean

In [93]:
input_dim = X_train.shape[1]

In [94]:
# Here's a Deep Dumb MLP
model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [95]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [99]:
model.fit(X_train, Y_train, nb_epoch=64, batch_size=128, validation_split=0.3)

Train on 1127 samples, validate on 484 samples
Epoch 1/64
1127/1127 [==============================] - 0s - loss: 6.7484e-04 - val_loss: 3.9258
Epoch 2/64
1127/1127 [==============================] - 0s - loss: 5.3705e-05 - val_loss: 3.9340
Epoch 3/64
 128/1127 [==>...........................] - ETA: 0s - loss: 1.5591e-04

/Users/taocheng/anaconda/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


1127/1127 [==============================] - 0s - loss: 1.2301e-04 - val_loss: 3.9652
Epoch 4/64
1127/1127 [==============================] - 0s - loss: 2.7797e-04 - val_loss: 3.9603
Epoch 5/64
1127/1127 [==============================] - 0s - loss: 2.3508e-04 - val_loss: 3.9823
Epoch 6/64
1127/1127 [==============================] - 0s - loss: 4.8570e-05 - val_loss: 3.9929
Epoch 7/64
1127/1127 [==============================] - 0s - loss: 4.9596e-05 - val_loss: 4.0118
Epoch 8/64
1127/1127 [==============================] - 0s - loss: 3.1935e-04 - val_loss: 4.0539
Epoch 9/64
1127/1127 [==============================] - 0s - loss: 1.0682e-04 - val_loss: 4.0353
Epoch 10/64
1127/1127 [==============================] - 0s - loss: 6.1956e-05 - val_loss: 4.0721
Epoch 11/64
1127/1127 [==============================] - 0s - loss: 4.9910e-05 - val_loss: 4.0718
Epoch 12/64
1127/1127 [==============================] - 0s - loss: 4.0644e-05 - val_loss: 4.1032
Epoch 13/64
1127/1127 [===============

In [101]:

preds14 = model.predict_classes(X_test, verbose=0)
acc14 = accuracy_score(test["Label"], preds14)

In [102]:
print('prediction accuracy: ', acc14)

prediction accuracy:  0.547619047619


In [103]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
nb_classes = 2

In [104]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(trainheadlines)
sequences_train = tokenizer.texts_to_sequences(trainheadlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)

/Users/taocheng/anaconda/lib/python3.6/site-packages/keras/preprocessing/text.py:139: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [105]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (1611, 200)
X_test shape: (378, 200)


In [106]:
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

/Users/taocheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/Users/taocheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [107]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [109]:
model.fit(X_train, Y_train, 
          batch_size=64, 
          nb_epoch=64,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)

Train on 1611 samples, validate on 378 samples
Epoch 1/64


/Users/taocheng/anaconda/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


1611/1611 [==============================] - 29s - loss: 0.0509 - acc: 0.9888 - val_loss: 1.2973 - val_acc: 0.5503
Epoch 2/64
1611/1611 [==============================] - 30s - loss: 0.0192 - acc: 0.9975 - val_loss: 1.4476 - val_acc: 0.5370
Epoch 3/64
1611/1611 [==============================] - 30s - loss: 0.0083 - acc: 0.9994 - val_loss: 1.5153 - val_acc: 0.5317
Epoch 4/64
1611/1611 [==============================] - 30s - loss: 0.0046 - acc: 1.0000 - val_loss: 1.5206 - val_acc: 0.5370
Epoch 5/64
1611/1611 [==============================] - 30s - loss: 0.0028 - acc: 1.0000 - val_loss: 1.7958 - val_acc: 0.5423
Epoch 6/64
1611/1611 [==============================] - 28s - loss: 0.0024 - acc: 0.9994 - val_loss: 1.6088 - val_acc: 0.5423
Epoch 7/64
1611/1611 [==============================] - 29s - loss: 0.0022 - acc: 1.0000 - val_loss: 1.8645 - val_acc: 0.5450
Epoch 8/64
1611/1611 [==============================] - 29s - loss: 0.0016 - acc: 1.0000 - val_loss: 1.8233 - val_acc: 0.5423
Epo

In [110]:
print('Test score:', score)
print('Test accuracy:', acc)


Test score: 2.54180836362
Test accuracy: 0.550264549949


In [111]:

preds15 = model.predict_classes(X_test, verbose=0)
acc15 = accuracy_score(test['Label'], preds15)

In [112]:
print('prediction accuracy: ', acc15)

prediction accuracy:  0.550264550265


In [113]:
# training model looks good, we can do some regularization & dropout if this is overfitting